In [1]:
import gc
import os
import logging
import datetime
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import lightgbm as lgb
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import StratifiedKFold
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_rows', 800)
pd.set_option('display.max_columns', 100)

import gc
import time
from tqdm import tqdm

In [3]:
%%time
PATH="../input/ltfs-fin-model/"
train = pd.read_csv(PATH+"train_preproc.csv",index_col=0)
test = pd.read_csv(PATH+"test_preproc.csv",index_col=0)
# subm_df = pd.read_csv(PATH+"sample_submission.csv")

CPU times: user 2.22 s, sys: 416 ms, total: 2.64 s
Wall time: 2.65 s


In [4]:
targetcol = 'loan_default'
target = train[targetcol]
train.shape, test.shape

((233154, 52), (112392, 52))

In [5]:
train.head()

,AVERAGE.ACCT.AGE,Aadhar_flag,CREDIT.HISTORY.LENGTH,Current_pincode_ID,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,Date.of.Birth,DisbursalDate,Driving_flag,Employee_code_ID,Employment.Type,MobileNo_Avl_Flag,NEW.ACCTS.IN.LAST.SIX.MONTHS,NO.OF_INQUIRIES,PAN_flag,PERFORM_CNS.SCORE,PERFORM_CNS.SCORE.DESCRIPTION,PRI.ACTIVE.ACCTS,PRI.CURRENT.BALANCE,PRI.DISBURSED.AMOUNT,PRI.NO.OF.ACCTS,PRI.OVERDUE.ACCTS,PRI.SANCTIONED.AMOUNT,PRIMARY.INSTAL.AMT,Passport_flag,SEC.ACTIVE.ACCTS,SEC.CURRENT.BALANCE,SEC.DISBURSED.AMOUNT,SEC.INSTAL.AMT,SEC.NO.OF.ACCTS,SEC.OVERDUE.ACCTS,SEC.SANCTIONED.AMOUNT,State_ID,UniqueID,VoterID_flag,asset_cost,branch_id,disbursed_amount,loan_default,ltv,manufacturer_id,supplier_id,date_of_birth,age,disbursal_date,disbursal_month,disbursal_year,disbursal_day,disbursal_dayofweek,PERFORM_CNS.SCORE.CATEGORY,Employment.Type.Category,AVERAGE.ACCT.AGE_MONTHS,CREDIT.HISTORY.LENGTH_MONTHS
0,0yrs 0mon,1,0yrs 0mon,1441,0,01-01-84,03-08-18,0,1998,Salaried,1,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,420825,0,58400,67,50578,0.0,89.55,45,22807,1984-01-01,35.279945,2018-08-03,8,2018,3,4,0,0,0,0
1,1yrs 11mon,1,1yrs 11mon,1502,1,31-07-85,26-09-18,0,1998,Self employed,1,0,0,0,598,I-Medium Risk,1,27600,50200,1,1,50200,1991,0,0,0,0,0,0,0,0,6,537409,0,65550,67,47145,1.0,73.23,45,22807,1985-07-31,33.700205,2018-09-26,9,2018,26,2,1,1,23,23
2,0yrs 0mon,1,0yrs 0mon,1497,0,24-08-85,01-08-18,0,1998,Self employed,1,0,0,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,417566,0,61360,67,53278,0.0,89.63,45,22807,1985-08-24,33.634497,2018-08-01,8,2018,1,2,0,1,0,0
3,0yrs 8mon,1,1yrs 3mon,1501,0,30-12-93,26-10-18,0,1998,Self employed,1,0,1,0,305,L-Very High Risk,0,0,0,3,0,0,31,0,0,0,0,0,0,0,0,6,624493,0,66113,67,57513,1.0,88.48,45,22807,1993-12-30,25.284052,2018-10-26,10,2018,26,4,2,1,8,15
4,0yrs 0mon,1,0yrs 0mon,1495,0,09-12-77,26-09-18,0,1998,Self employed,1,0,1,0,0,No Bureau History Available,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,539055,0,60300,67,52378,1.0,88.39,45,22807,1977-12-09,41.341547,2018-09-26,9,2018,26,2,0,1,0,0


Target Encoding Functions

In [6]:
def getenccolname(colname,cols_agg):
    if 'var' in cols_agg:
        colname ="targetvarenc_"+colname
    elif 'std' in cols_agg:
        colname ="targetstdenc_"+colname 
    elif 'sum' in cols_agg:   
        colname ="targetsumenc_"+colname
    elif 'min' in cols_agg:   
        colname ="targetminenc_"+colname
    elif 'max' in cols_agg:   
        colname ="targetmaxenc_"+colname
    elif 'median' in cols_agg:   
        colname ="targetmedianenc_"+colname
    elif 'count' in cols_agg:   
        colname ="targetcountenc_"+colname
    elif 'iqmean' in cols_agg:   
        colname ="targetiqmeanenc_"+colname
    else:
        colname ="targetenc_"+colname
        
    return colname

def droptargetenccols(train, val,test):
     #remove target encoding fields if present
    targetenccols = [col for col in train.columns if ('targetenc' in col) or ('targetstdenc' in col)]
    train.drop(targetenccols,axis=1,inplace=True)
    val.drop(targetenccols,axis=1,inplace=True)
    targetenccols_test = [col for col in test.columns if ('targetenc' in col) or ('targetstdenc' in col)]
    test.drop(targetenccols_test,axis=1,inplace=True)
    
    return train, val,test

In [7]:
def impact_coding(data, feature, target, n_folds=20, n_inner_folds=10):
    from sklearn.model_selection import StratifiedKFold
    '''
    ! Using oof_default_mean for encoding inner folds introduces leak.
    
    Source: https://www.kaggle.com/tnarik/likelihood-encoding-of-categorical-features
    
    Changelog:    
    a) Replaced KFold with StratifiedFold due to class imbalance
    b) Rewrote .apply() with .map() for readability
    c) Removed redundant apply in the inner loop
    d) Removed global average; use local mean to fill NaN values in out-of-fold set
    '''
    impact_coded = pd.Series()
        
    kf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=8888) # KFold in the original
    oof_mean_cv = pd.DataFrame()
    split = 0
    print('col:',feature)
    print()
    for outer_, (infold, oof) in enumerate(kf.split(data[feature], data[target])):
        print('outer fold:{0} '.format(outer_))

        kf_inner = StratifiedKFold(n_splits=n_inner_folds, shuffle=True, random_state=8888)
        inner_split = 0
        inner_oof_mean_cv = pd.DataFrame()
        oof_default_inner_mean = data.iloc[infold][target].mean()
        
        tr_outer = data.iloc[infold]
        
        for inner_,(infold_inner, oof_inner) in enumerate(kf_inner.split(data.iloc[infold], data.loc[infold, target])):
                    
            # The mean to apply to the inner oof split (a 1/n_folds % based on the rest)
#             oof_mean = data.iloc[infold_inner].groupby(by=feature)[target].mean()
            oof_mean = tr_outer.iloc[infold_inner].groupby(by=feature)[target].mean()
            
            # Also populate mapping (this has all group -> mean for all inner CV folds)
            inner_oof_mean_cv = inner_oof_mean_cv.join(pd.DataFrame(oof_mean), rsuffix=inner_split, how='outer')
            inner_oof_mean_cv.fillna(value=oof_default_inner_mean, inplace=True)
            inner_split += 1

        # compute mean for each value of categorical value across oof iterations
        inner_oof_mean_cv_map = inner_oof_mean_cv.mean(axis=1)

        # Also populate mapping
        oof_mean_cv = oof_mean_cv.join(pd.DataFrame(inner_oof_mean_cv), rsuffix=split, how='outer')
        oof_mean_cv.fillna(value=oof_default_inner_mean, inplace=True) # <- local mean as default
        split += 1

        feature_mean = data.loc[oof, feature].map(inner_oof_mean_cv_map).fillna(oof_default_inner_mean)
        impact_coded = impact_coded.append(feature_mean)
    
    oof_default_mean = data[target].mean() # Gobal mean to use by default (you could further tune this)
    return impact_coded, oof_mean_cv.mean(axis=1), oof_default_mean



def encode_target_cv(tr, val, test, targetcolname, categ_variables, impact_coder=impact_coding,
                    n_folds=20, n_inner_folds=10):
    """Apply original function for each <categ_variables> in  <data>
    Reduced number of validation folds
    """
    train_target = tr.copy() 
    
    code_map = dict()
    default_map = dict()
    for f in categ_variables:
        enccol_mean = getenccolname(f,'mean')
        train_target.loc[:, enccol_mean], code_map[f], default_map[f] = impact_coder(train_target, f, 
                                                                                     targetcolname,
                                                                                    n_folds=n_folds, 
                                                                                     n_inner_folds=n_inner_folds)
        val.loc[:, enccol_mean] = val[f].map(code_map[f]).fillna(default_map[f])
        test.loc[:, enccol_mean] = test[f].map(code_map[f]).fillna(default_map[f])
        
        
#     return train_target, code_map, default_map
    return train_target,val,test

In [8]:
def performsmoothing(averages,targetcolname,train,agg,countSeries,
                     smoothing,min_samples_leaf,noise_level,global_agg_val=None):
    
#         smoothing_v = 1 / (1 + np.exp(-((averages["count"] ) - min_samples_leaf) / smoothing))
    smoothing_v = 1 / (1 + np.exp(-((countSeries - min_samples_leaf) / smoothing)) )

#         print('averages[count] describe:',averages["count"].describe())
    print('smoothing_v describe:',smoothing_v.describe())
    if agg=='mean':
#             global_agg_val = train[targetcolname].mean()
#             print('train[targetcolname] describe:',train[targetcolname].describe())
        if global_agg_val is None:
            global_agg_val = np.nanmean(train[targetcolname].values)
    elif agg=='std':
        global_agg_val= train[targetcolname].std()

    newcol ='newcol'
    if agg=='std':
        print('std before smoothing:',averages[agg].head(25))

    print('averages[agg].shape:',averages[agg].shape)
    print('smoothing_v.shape:',smoothing_v.shape)
#         print('global_agg_val:',global_agg_val)

    averages[newcol] = global_agg_val * (1 - smoothing_v) + averages[agg] * smoothing_v
    
#     averages[newcol] = (countSeries * averages[agg] + smoothing * global_agg_val) / (countSeries + smoothing)
#         smooth = (counts * means + m * mean) / (counts + m)


    np.random.seed(42)
    noise = np.random.randn(len(averages[newcol])) * noise_level
    averages[newcol] = averages[newcol] + noise

    if agg=='std':
        print('std after smoothing:',averages[newcol].head(25))

    del smoothing_v,noise;gc.collect()

    return averages[newcol]

 
def targetencode(train,val,test,catcolnames,targetcolname,
                         smoothing,min_samples_leaf,noise_level):
    #Target Encoding
    
    for i,curcol in enumerate(catcolnames):
        print()
        print('curcol: ',curcol)
        print()
        
        enccol_mean = getenccolname(curcol,'mean')
        enccol_count = getenccolname(curcol,'count')
        
#         print('enccol_mean:',enccol_mean)
#         print('enccol_count:',enccol_count)
        
#         train['group_count'] = train.groupby([curcol]).transform('count')
        averages = train.groupby(curcol).agg({targetcolname: "mean",
                                              curcol: "count"})
#         averages = train.groupby(curcol).agg({targetcolname: "mean",
#                                               'group_count': "first"})

        averages.columns =['mean',enccol_count]
#         print('averages count quantile 0.95:',averages[enccol_count].quantile(0.05))
#         print('averages count describe:',averages[enccol_count].describe())
#         print('averages count quantile 0.95:',averages[enccol_count].quantile(0.95))
        print()
        print('averages mean quantile 0.95:',averages['mean'].quantile(0.05))
        print('averages mean describe:',averages['mean'].describe())
        print('averages mean quantile 0.95:',averages['mean'].quantile(0.95))
        start = time.time()
    
        averages[enccol_mean] = performsmoothing(averages,targetcolname,train,'mean',averages[enccol_count],
                                          smoothing[i],min_samples_leaf[i],noise_level[i],
                                          global_agg_val=None)
        averages.drop(['mean'],axis=1,inplace=True)
        
        train[enccol_mean] = train[curcol].map(averages[enccol_mean])
        val[enccol_mean] = val[curcol].map(averages[enccol_mean])
        test[enccol_mean] = test[curcol].map(averages[enccol_mean])
        
#         del train['group_count'];gc.collect()
        
        end = time.time()
        print('update exec time:',end- start)
        
        del averages;gc.collect()
#         print(train[train[enccol_mean].isnull()].shape)
        print('val null:',val[val[enccol_mean].isnull()].shape)
        print('test null:',test[test[enccol_mean].isnull()].shape)
#         print('test unique count:',test.loc[test[enccol_mean].isnull(),curcol].nunique())
#         test_missing = test.loc[test[enccol_mean].isnull(),curcol].unique()
#         print('test unique null:',test_missing)
        
#         mask = val[curcol].isin(test_missing)
#         val_temp = val[mask]
#         print('val of test missing count;',val_temp[curcol].nunique())
#         print('val test missing value counts:',val_temp[curcol].value_counts())
        
#         global_mean= train[enccol_mean].mean()
#         train[enccol_mean].fillna(global_mean,inplace=True)
#         val[enccol_mean].fillna(global_mean,inplace=True)
#         test[enccol_mean].fillna(global_mean,inplace=True)

         
    return train,val,test
   

In [9]:
import time
def runlgb(ispermutefeats,train,test,param,cur_features,score_function=None):

    overall_sel_feats =[]
    overall_imp_df = pd.DataFrame()
    overall_imp_df['feature']= np.array(cur_features)
    overall_imp_df['overall_score_mean'] =0 
    overall_imp_df['overall_score_max'] =-9999 
    overall_imp_df['overall_score_min'] =9999 
    
    oof = np.zeros(len(train))
    predictions = np.zeros(len(test))
    start = time.time()
    valid_scores =[]
    fold_importance_df = pd.DataFrame()
    

    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=4590)
    indices = folds.split(train.values, target.values)
        
    for fold_, (trn_idx, val_idx) in enumerate(indices):
        print()
        print("fold n°{}".format(fold_))

        tr = train.iloc[trn_idx]
        val = train.iloc[val_idx]
        y_val = target.iloc[val_idx]
        y_tr = target.iloc[trn_idx]
        
        #Concat current encoding train, valid and test files
        cur_tr_encs = tr_encs[fold_]
        cur_val_encs= val_encs[fold_]

        print('val shape bef:',val.shape)
        print('tr shape bef:',tr.shape)
        print('test shape bef:',test.shape)
        
        tr=pd.concat([tr,cur_tr_encs],axis=1)
        val=pd.concat([val,cur_val_encs],axis=1)
        test_cur=pd.concat([test,test_encs[fold_]],axis=1)
        
#         targetenccols = ['targetenc_branch_id','targetenc_manufacturer_id','targetenc_State_ID',
#                          'targetenc_Current_pincode_ID','targetenc_Employee_code_ID','targetenc_supplier_id']
#         targetenccols = ['targetenc_Current_pincode_ID','targetenc_Employee_code_ID','targetenc_supplier_id']

#         tr['targetenc_mean'] = tr[targetenccols].mean(axis=1)
#         val['targetenc_mean'] = val[targetenccols].mean(axis=1)        
#         test_cur['targetenc_mean'] = test_cur[targetenccols].mean(axis=1)        
        
        print('val shape after:',val.shape)
        print('tr shape after:',tr.shape)
        print('test shape after:',test.shape)
         
#         val_index_ser = pd.Series(np.array(val.index))
#         print('val shape:',val.shape)
#         print('val index head:',val_index_ser.head(20))
#         print('val index tail:',val_index_ser.tail(20))
        
        trn_data = lgb.Dataset(tr[cur_features], label=y_tr)#,, categorical_feature=categorical_feats)
        val_data = lgb.Dataset(val[cur_features], label=y_val)#,, categorical_feature=categorical_feats)
        
        clf = lgb.train(param, trn_data, num_round, valid_sets = [val_data], verbose_eval=500, 
                        early_stopping_rounds = 300)

        #Prediction based on current fold selected features
        if ispermutefeats:
            
            selected_features, importance_df = permutation_feature_selection(clf, val[cur_features], 
                                                                             y_val,score_function,
                                                                             rep=4,max_delta_score=max_delta_score)
            overall_sel_feats += [selected_features]
            print(selected_features)

#             print('overal imp shape:{0} importance_df shape:{1}'.format(overall_imp_df.shape,importance_df.shape))
            
            overall_imp_df['fold_'+str(fold_)+'score_mean'] = importance_df['delta_score_mean']
            overall_imp_df['fold_'+str(fold_)+'score_max'] = importance_df['delta_score_max']
            overall_imp_df['fold_'+str(fold_)+'score_min'] = importance_df['delta_score_min']
        else:
            oof[val_idx] = clf.predict(val[cur_features], num_iteration=clf.best_iteration)

            fold_importance_df["feature"] = cur_features
            if fold_==0:
                fold_importance_df["importance"] =0
            fold_importance_df["importance"] += clf.feature_importance() / n_splits
            valid_scores+=[clf.best_score['valid_0'][param['metric']]]
            predictions += clf.predict(test_cur[cur_features], num_iteration=clf.best_iteration) / folds.n_splits

    if ispermutefeats:
        fold_mean_cols = [col for col in overall_imp_df.columns if ('score_mean' in col) and ('fold_' in col) ]
        fold_max_cols = [col for col in overall_imp_df.columns if ('score_max' in col) and ('fold_' in col) ]
        fold_min_cols = [col for col in overall_imp_df.columns if ('score_min' in col) and ('fold_' in col) ]
        overall_imp_df['overall_score_mean'] = overall_imp_df[fold_mean_cols].mean(axis=1)
        overall_imp_df['overall_score_max'] = overall_imp_df[fold_max_cols].max(axis=1)
        overall_imp_df['overall_score_min'] = overall_imp_df[fold_min_cols].min(axis=1)
    else:
        print('valid scores:',valid_scores)
        print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

    return fold_importance_df,predictions,oof,overall_imp_df,overall_sel_feats

Model

In [10]:
# exclude_cols =['Date.of.Birth','Employment.Type','DisbursalDate',
#                'PERFORM_CNS.SCORE.DESCRIPTION','AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH',
#                'MobileNo_Avl_Flag','disbursal_year','disbursal_day','disbursal_dayofweek',
#                'date_of_birth', 'disbursal_date',
#                'UniqueID',targetcol]
# features = [c for c in train.columns if c not in exclude_cols]
# print(features)

In [11]:
# param = {'num_leaves': 31,
#          'min_data_in_leaf': 30, 
#          'objective':'binary',
#          'max_depth': -1,
#          'learning_rate': 0.01,
#          "min_child_samples": 20,
#          "boosting": "gbdt",
#          "feature_fraction": 0.9,
#          "bagging_freq": 1,
#          "bagging_fraction": 0.9 ,
#          "bagging_seed": 11,
#          "metric": 'auc',
#          "lambda_l1": 0.1,
#          "verbosity": -1,
#          "nthread": 4,
#          'n_estimators' : 10000,
#          "random_state": 4590}

In [12]:
n_splits=5
num_round = 10
max_delta_score =0.0001

In [13]:

catcolnames =[
             'branch_id',
#     'manufacturer_id',      'State_ID',
#              'PERFORM_CNS.SCORE.CATEGORY'
             'Current_pincode_ID', 'Employee_code_ID', 'supplier_id',
             ]


# min_samples_leaf=[2000,2000,2000,2000,2000,2000]
# smoothing=[10,10,10,10,10,10]
# noise_level=[0.1,0.1,0.1,0.1,0.1,0.1]

# min_samples_leaf=[2000,2000,2000]
# smoothing=[10,10,10]
# noise_level=[0.1,0.1,0.1]

# min_samples_leaf=[200,200,200]
# smoothing=[20,20,20]
# noise_level=[0.1,0.1,0.1]

In [14]:
param = {'colsample_bytree': 0.7196484570790651,
   'min_child_samples': 235,
   'num_leaves': 36,
   'reg_alpha': 0.6474702076362333,
   'reg_lambda': 0.021458900986429996,
   'subsample': 0.8873887256306612,
   'subsample_for_bin': 110000,
   'learning_rate': 0.01,
   'boosting': 'gbdt',
   'bagging_seed': 2018,
   'bagging_freq': 2,
   'min_data_in_bin': 100,
   'n_estimators': 10000,
   'objective': 'binary',
   'metric': 'auc',
   'random_state': 2333,
   'max_depth': 15,
   'scale_pos_weight': 1}

In [15]:
def gen_targetencode(train,target,test,n_splits,catcolnames,targetcol,
                    smoothing,min_samples_leaf,noise_level):

    start = time.time()
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=4590)
    indices = folds.split(train.values, target.values)

    test_index = test.index

    for fold_, (trn_idx, val_idx) in enumerate(indices):
        print('******************************************************')
        print("FOLD  ---  {}".format(fold_))
        print('******************************************************')

        tr = train.iloc[trn_idx]
        val = train.iloc[val_idx]
        
        tr_index = tr.index
        val_index = val.index
        #drop any existing target enc cols
        tr,val,test = droptargetenccols(tr,val,test)

#         #target encoding on transaction merchant id
#         tr,val,test = targetencode(tr, val,test,catcolnames,targetcol,
#                                     smoothing,min_samples_leaf,noise_level)

        tr,val,test = encode_target_cv(tr.reset_index(drop=True), val.reset_index(drop=True), 
                                       test.reset_index(drop=True), 
                                       targetcol, catcolnames,
                                       n_folds=20, n_inner_folds=10
                                      )

        tr.index = tr_index
        val.index = val_index
        test.index = test_index


        enc_cols = [col for col in tr.columns if 'targetenc' in col]
        print('enc cols:',enc_cols)
        print('save encoding feats...')

        #save target encoding features in separate file
        tr[enc_cols].to_hdf('train_targetenc_feats'+str(fold_)+'.hdf',key='data')
        val[enc_cols].to_hdf('val_targetenc_feats'+str(fold_)+'.hdf',key='data')
        test[enc_cols].to_hdf('test_targetenc_feats'+str(fold_)+'.hdf',key='data')

        print('save encoding feats in csv...')
        tr[enc_cols].to_csv('train_targetenc_feats'+str(fold_)+'.csv')
        val[enc_cols].to_csv('val_targetenc_feats'+str(fold_)+'.csv')
        test[enc_cols].to_csv('test_targetenc_feats'+str(fold_)+'.csv')


    end = time.time()
    print('Target Enc Execution Time:',end-start)

In [16]:
# gen_targetencode(train,target,test,n_splits,catcolnames,targetcol,
#                     smoothing,min_samples_leaf,noise_level)

In [17]:
def getenc():
    tr_encs = []
    val_encs = []
    test_encs = []
    
    Path=''

    for i in range(0,5):
        cur_tr_enc = pd.read_csv(Path+'train_targetenc_feats'+str(i)+'.csv',index_col=0)
        cur_val_enc = pd.read_csv(Path+'val_targetenc_feats'+str(i)+'.csv',index_col=0)

        tr_encs += [cur_tr_enc]
        val_encs +=[ cur_val_enc]

        test_encs += [pd.read_csv(Path+'test_targetenc_feats'+str(i)+'.csv',index_col=0)]
        print('read complete for:',i)
        
    return tr_encs,val_encs,test_encs

In [18]:
# # tr_encs, val_encs,test_encs = getenc(mask_without_outlier=mask_without_outlier)
# tr_encs, val_encs,test_encs = getenc()
# print(tr_encs[0].shape)
# print(val_encs[0].shape)
# print(test_encs[0].shape)

In [19]:
# targetenccols = ['targetenc_branch_id','targetenc_manufacturer_id','targetenc_State_ID',
#                  'targetenc_Current_pincode_ID','targetenc_Employee_code_ID','targetenc_supplier_id']

# train['targetenc_mean'] = train[targetenccols].mean(axis=1)
# test['targetenc_mean'] = test[targetenccols].mean(axis=1)

In [20]:
# sel_enc_cols = [col for col in test.columns if 'targetenc_' in col]
# print(sel_enc_cols)
sel_enc_cols= ['targetenc_Current_pincode_ID','targetenc_Employee_code_ID','targetenc_supplier_id',
              'targetenc_branch_id']
# sel_enc_cols= ['targetenc_branch_id']
exclude_cols =['Date.of.Birth','Employment.Type','DisbursalDate',
               'PERFORM_CNS.SCORE.DESCRIPTION','AVERAGE.ACCT.AGE','CREDIT.HISTORY.LENGTH',
               'MobileNo_Avl_Flag','disbursal_year','disbursal_day','disbursal_dayofweek',
               'date_of_birth', 'disbursal_date',
               'UniqueID',targetcol,
#               'branch_id', 'manufacturer_id', 'State_ID', 'Current_pincode_ID', 
#                'Employee_code_ID', 'supplier_id'
              ]
features = [c for c in train.columns if c not in exclude_cols]
features +=sel_enc_cols
# features +=['targetenc_mean']

# targetenccols = ['targetenc_branch_id','targetenc_manufacturer_id','targetenc_State_ID',
#                  'targetenc_Current_pincode_ID','targetenc_Employee_code_ID','targetenc_supplier_id']

print(features)

['Aadhar_flag', 'Current_pincode_ID', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'Driving_flag', 'Employee_code_ID', 'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'NO.OF_INQUIRIES', 'PAN_flag', 'PERFORM_CNS.SCORE', 'PRI.ACTIVE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.DISBURSED.AMOUNT', 'PRI.NO.OF.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.SANCTIONED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'Passport_flag', 'SEC.ACTIVE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.DISBURSED.AMOUNT', 'SEC.INSTAL.AMT', 'SEC.NO.OF.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.SANCTIONED.AMOUNT', 'State_ID', 'VoterID_flag', 'asset_cost', 'branch_id', 'disbursed_amount', 'ltv', 'manufacturer_id', 'supplier_id', 'age', 'disbursal_month', 'PERFORM_CNS.SCORE.CATEGORY', 'Employment.Type.Category', 'AVERAGE.ACCT.AGE_MONTHS', 'CREDIT.HISTORY.LENGTH_MONTHS', 'targetenc_Current_pincode_ID', 'targetenc_Employee_code_ID', 'targetenc_supplier_id', 'targetenc_branch_id']


In [21]:
tr_encs=None; val_encs=None;test_encs=None;

In [22]:
def runlgb_with_targetenc(train,test,param,features,
                         n_splits,catcolnames,targetcol,
                    smoothing,min_samples_leaf,noise_level):
    
    global tr_encs, val_encs,test_encs

    gen_targetencode(train,target,test,n_splits,catcolnames,targetcol,
                    smoothing,min_samples_leaf,noise_level)

    tr_encs, val_encs,test_encs = getenc()
    print(tr_encs[0].shape)
    print(val_encs[0].shape)
    print(test_encs[0].shape)
    
    fold_importance_df,predictions,oof,overall_imp_df,overall_sel_feats = \
        runlgb(False,train ,test,param,features,score_function=None)
    
    return fold_importance_df,predictions,oof

In [23]:
# %%time
# min_samples_leaf=[500,500,500]
# smoothing=[50,50,50]
# noise_level=[0.1,0.1,0.1]


# fold_importance_df,predictions,oof = runlgb_with_targetenc(train,test,param,features,
#                                      n_splits,catcolnames,targetcol,
#                                 smoothing,min_samples_leaf,noise_level)

# sub_df = pd.DataFrame({"UniqueID":test["UniqueID"].values})
# sub_df[targetcol] = predictions
# sub_df.to_csv("submission_targetenc_100L_10S.csv", index=False)
# np.save('oof_targetenc_100L_10S.npy',oof)
# oof1 = oof.copy()
# predictions1 = predictions.copy()

In [24]:
# %%time
# min_samples_leaf=[1000,1000,1000]
# smoothing=[100,100,100]
# noise_level=[0.1,0.1,0.1]

# fold_importance_df,predictions,oof = runlgb_with_targetenc(train,test,param,features,
#                                      n_splits,catcolnames,targetcol,
#                                 smoothing,min_samples_leaf,noise_level)

# sub_df = pd.DataFrame({"UniqueID":test["UniqueID"].values})
# sub_df[targetcol] = predictions
# sub_df.to_csv("submission_targetenc_100L_0.5S.csv", index=False)
# np.save('oof_targetenc_100L_0.5S.npy',oof)
# oof2 = oof.copy()
# predictions2 = predictions.copy()

In [25]:
%%time

param = {'colsample_bytree': 0.7196484570790651,
   'min_child_samples': 235,
   'num_leaves': 36,
   'reg_alpha': 0.6474702076362333,
   'reg_lambda': 0.021458900986429996,
   'subsample': 0.8873887256306612,
   'subsample_for_bin': 110000,
   'learning_rate': 0.01,
   'boosting': 'gbdt',
   'bagging_seed': 2018,
   'bagging_freq': 2,
   'min_data_in_bin': 100,
   'n_estimators': 10000,
   'objective': 'binary',
   'metric': 'auc',
   'random_state': 2333,
   'max_depth': 15,
   'scale_pos_weight': 1}

min_samples_leaf=[1000,1000,1000]
smoothing=[100,100,100]
noise_level=[0.1,0.1,0.1]

fold_importance_df,predictions,oof = runlgb_with_targetenc(train,test,param,features,
                                     n_splits,catcolnames,targetcol,
                                smoothing,min_samples_leaf,noise_level)

sub_df = pd.DataFrame({"UniqueID":test["UniqueID"].values})
sub_df[targetcol] = predictions
sub_df.to_csv("submission_targetenc_300L_without_fillNA.csv", index=False)
np.save('oof_targetenc_300L_without_fillNA.npy',oof)
oof3 = oof.copy()
predictions3 = predictions.copy()

******************************************************
FOLD  ---  0
******************************************************
col: branch_id

outer fold:0 
outer fold:1 
outer fold:2 
outer fold:3 
outer fold:4 
outer fold:5 
outer fold:6 
outer fold:7 
outer fold:8 
outer fold:9 
outer fold:10 
outer fold:11 
outer fold:12 
outer fold:13 
outer fold:14 
outer fold:15 
outer fold:16 
outer fold:17 
outer fold:18 
outer fold:19 
col: Current_pincode_ID

outer fold:0 
outer fold:1 
outer fold:2 
outer fold:3 
outer fold:4 
outer fold:5 
outer fold:6 
outer fold:7 
outer fold:8 
outer fold:9 
outer fold:10 
outer fold:11 
outer fold:12 
outer fold:13 
outer fold:14 
outer fold:15 
outer fold:16 
outer fold:17 
outer fold:18 
outer fold:19 
col: Employee_code_ID

outer fold:0 
outer fold:1 
outer fold:2 
outer fold:3 
outer fold:4 
outer fold:5 
outer fold:6 
outer fold:7 
outer fold:8 
outer fold:9 
outer fold:10 
outer fold:11 
outer fold:12 
outer fold:13 
outer fold:14 
outer fold:15 
out

In [26]:
# # %%time

# param = {'num_leaves': 31,
#          'min_data_in_leaf': 30, 
#          'objective':'binary',
#          'max_depth': -1,
#          'learning_rate': 0.01,
#          "min_child_samples": 20,
#          "boosting": "gbdt",
#          "feature_fraction": 0.9,
#          "bagging_freq": 1,
#          "bagging_fraction": 0.9 ,
#          "bagging_seed": 11,
#          "metric": 'auc',
#          "lambda_l1": 0.1,
#          "verbosity": -1,
#          "nthread": 4,
#          'n_estimators' : 10000,
#          "random_state": 4590}


# min_samples_leaf=[300,300,300]
# smoothing=[30,30,30]
# noise_level=[0.1,0.1,0.1]

# fold_importance_df,predictions,oof = runlgb_with_targetenc(train,test,param,features,
#                                      n_splits,catcolnames,targetcol,
#                                 smoothing,min_samples_leaf,noise_level)

# sub_df = pd.DataFrame({"UniqueID":test["UniqueID"].values})
# sub_df[targetcol] = predictions
# sub_df.to_csv("submission_targetenc_300L_with_normalparam.csv", index=False)
# np.save('oof_targetenc_300L_with_normalparam.npy',oof)
# oof2 = oof.copy()
# predictions2 = predictions.copy()

In [27]:
# %%time
# fold_importance_df,predictions,oof,overall_imp_df,overall_sel_feats = \
#         runlgb(False,train ,test,param,features,score_function=None)

In [28]:
# oof1 = oof.copy() #min_samples_leaf=[12,62,36]  smoothing=[1,1,1]  new param
# oof2 = oof.copy() # min_samples_leaf=[100,100,100]  smoothing=[10,10,10] old param 
# oof3 = oof.copy() # min_samples_leaf=[100,100,100]  smoothing=[10,10,10] new param
# oof4 = oof.copy() #min_samples_leaf=[200,200,200] smoothing=[5,5,5]

In [29]:
oof = oof1 + oof2 + oof3 
print('Ens AUC:',roc_auc_score(target,oof))
predictions = predictions1 + predictions2 + predictions3

NameError: name 'oof1' is not defined

In [30]:
sub_df = pd.DataFrame({"UniqueID":test["UniqueID"].values})
sub_df[targetcol] = predictions
sub_df.to_csv("submission_targetenc_ens.csv", index=False)
np.save('oof_targetenc_ens.npy',oof)

In [31]:
# %%time
# PATH="../input/ltfs-fin/"
# train_orig = pd.read_csv(PATH+"train.csv")
# test_orig = pd.read_csv(PATH+"test.csv")
# # subm_df = pd.read_csv(PATH+"sample_submission.csv")

In [32]:
# catcolnames =[
# #     'branch_id','manufacturer_id','State_ID',
#              'Current_pincode_ID', 'Employee_code_ID', 'supplier_id',
# #              'PERFORM_CNS.SCORE.CATEGORY'
#              ]
# #sel_enc_cols
# for col in catcolnames:
#     print()
#     print('col:',col)
#     test_unique=test_orig[col].unique() 
#     train_unique=train_orig[col].unique() 
#     test_m_train = list(set(test_unique).difference(train_unique))
#     print((test_m_train))


In [33]:
# catcolnames =[
# #     'branch_id','manufacturer_id','State_ID',
#              'Current_pincode_ID', 'Employee_code_ID', 'supplier_id',
# #              'PERFORM_CNS.SCORE.CATEGORY'
#              ]
# #sel_enc_cols
# for col in catcolnames:
#     print()
#     print('col:',col)
#     print('*********Train***********')
#     df = pd.DataFrame()
#     df['train']=train[col].value_counts() 
#     df['train_freq']=df['train'] / df['train'].sum()
#     df['test']=test[col].value_counts()
#     df['test_freq']=df['test'] / df['test'].sum()
# #     df=train[col].value_counts().reset_index()
#     print(df.head(100))
#     print(df['train'].describe())
#     print(df['train'].quantile(0.95))
#     print()
#     print(df['test'].describe())
#     print(df['test'].quantile(0.95))
# #     print(train[col].value_counts())
#     print('*********TEST***********')
# #     print(test[col].value_counts())

In [34]:
# print(test['branch_id'].head(25))
# print(test_encs[0]['targetenc_branch_id'].head(25))
# print(test_encs[1]['targetenc_branch_id'].head(25))
# print(test_encs[2]['targetenc_branch_id'].head(25))
# print(test_encs[3]['targetenc_branch_id'].head(25))
# print(test_encs[4]['targetenc_branch_id'].head(25))